# 01: Vozidla hromadné dopravy IDS JMK

Na https://mapa.idsjmk.cz/ můžeš sledovat aktuální polohu a další informace o vozidlech Integrovaného dopravního systému Jihomoravského kraje. Data si zkusíme analyzovat:

- Otevři si https://mapa.idsjmk.cz/api/vehicles.json v prohlížeči a podívej se na strukturu dat, ať víš, do čeho jdeš:).

- Načti si data z API na adrese https://mapa.idsjmk.cz/api/vehicles.json pomocí modulu `requests`. (Kdyby nefungovalo API, využij přiložený soubor `vehicles.json`.) *Pozor, načtení pomocí `response.json()` vrací chybu kvůli skrytým nestandardním znakům. Proto využij modul `json`, kterým zpracuješ vrácenou response příkazem `data = json.loads(response.content.decode("utf-8-sig"))`.*

- Zobraz datum a čas vytvoření souboru ve formátu jako např. "Datum: 2024-12-31, čas: 08:30:00". Data najdeš pod klíčem 'LastUpdate'. Využij jednoduchý slicing (indexování) řetězce.

Následující úkoly můžeš řešit pomocí slovníků v pythonu nebo pomocí pandas. Ideální bude vyzkoušet obojí, ale potom si radši pro každé řešení vytvoř nový soubor nebo novou buňku v jupyter notebooku, ať se to nemíchá.

*Pro pandas variantu řešení načti jako DataFrame pouze podslovník uložený pod klíčem 'Vehicles'. Bude se hodit metoda `DataFrame.from_dict()`.*

1. Spočítej následující hodnoty a vypiš výsledek *(vyzkoušej si všechny nebo jen některé body)*:
- Počet všech vozidel
- Počet vozidel na pauze ('IsInactive' je `True`).
- Počet vozidel se zpožděním ('Delay') větším než 5 minut. Kolik je to procent ze všech vozidel?
- Průměr zpoždění všech vozidel (včetně těch pod 5 minut).
- Počet nízkopodlažních ('LF' je `True`) regionálních vlaků  ('LineName' obsahuje 'R' nebo 'S'). Pro pandas variantu řešení se na 'LineName' může hodit metoda `str.contains()`.

2. Spočítej zvlášť ~~tramvaje~~ šaliny ('LineID' je 1-12), trolejbusy ('LineID' je 20-39) a městské autobusy ('LineID' je 40-99). Aktivní a neaktivní není potřeba rozlišovat. *Pro pandas variantu řešení můžeš využít složené podmínky nebo metodu `between()`.* Výsledek vypiš a ulož do souboru `pocty_vozidel_brno.json` jako slovník/JSON s klíči "pocet_tramvaji", "pocet_trolejbusu" a "pocet_autobusu" a odpovídajícími hodnotami.

Pokud pracuješ s aktuálními daty, můžeš počty porovnat s těmi na adrese https://mapa.idsjmk.cz/ (nemusí sedět úplně přesně kvůli výlukovým linkám, ale ty neuvažujme).

*Podrobnější popis položek JSONu najdeš např. na [data.brno.cz/...](https://data.brno.cz/datasets/mestobrno::polohy-vozidel-hromadn%C3%A9-dopravy-public-transit-positional-data/about), kde se nachází i databáze s historií dat.*

In [6]:
import requests
import json

url = "https://mapa.idsjmk.cz/api/vehicles.json"

response = requests.get(url)
data = json.loads(response.content.decode("utf-8-sig"))
data

{'LastUpdate': '2024-04-24T21:00:32.3781021+02:00',
 'Vehicles': [{'ID': 1922,
   'IDB': 0,
   'IDC': 0,
   'VType': 0,
   'LType': 0,
   'Lat': 49.192734,
   'Lng': 16.572811,
   'Bearing': -1,
   'LineID': 2,
   'LineName': '2',
   'RouteID': 1505,
   'ServiceId': 212,
   'Course': '00212',
   'LF': True,
   'Delay': 0,
   'LastStopID': 9410,
   'FinalStopID': 1343,
   'FinalStopName': 'Lipová',
   'IsInactive': True},
  {'ID': 21571,
   'IDB': 0,
   'IDC': 0,
   'VType': 4,
   'LType': 4,
   'Lat': 48.853413,
   'Lng': 17.109982,
   'Bearing': 45,
   'LineID': 901,
   'LineName': '901',
   'RouteID': 75,
   'ServiceId': 90301,
   'Course': '90301',
   'LF': True,
   'Delay': 0,
   'LastStopID': 19019,
   'FinalStopID': 19008,
   'FinalStopName': 'Hodonín, nem.láz.',
   'IsInactive': False},
  {'ID': 21136,
   'IDB': 0,
   'IDC': 0,
   'VType': 4,
   'LType': 4,
   'Lat': 48.755608,
   'Lng': 16.850769,
   'Bearing': 225,
   'LineID': 571,
   'LineName': '571',
   'RouteID': 23,
   '

In [7]:
# Výpis data a času vytvoření
timestamp = data['LastUpdate']
# print(timestamp)
print(f"Datum: {timestamp[:10]}, čas: {timestamp[11:11+8]}")

Datum: 2024-04-24, čas: 21:00:32


## 1. Řešení se slovníky:

In [8]:
vehicles = data['Vehicles']

# Počet všech vozidel
pocet_vozidel = len(vehicles)

# Inicializace proměnných
hranice_maleho_zpozdeni = 5

pocet_neaktivnich = 0
pocet_nizkopodlaznich_regionalnich_vlaku = 0
pocet_zpozdenych_nad5min = 0
zpozdeni_celkem = 0
pocty_vozidel_brno = {
    'pocet_tramvaji': 0,
    'pocet_trolejbusu': 0,
    'pocet_autobusu': 0,
}

# Procházení slovníku a sbírání dat
for vehicle in vehicles:

    # Počítadlo neaktivních vozidel
    if vehicle['IsInactive']:
        pocet_neaktivnich += 1
    
    # Počítadlo zpožděných nad 5 minut
    if vehicle['Delay'] > hranice_maleho_zpozdeni:
        pocet_zpozdenych_nad5min += 1

    # Sčítadlo všech zpoždění
    zpozdeni_celkem += vehicle['Delay']

    # Počítání vozidel podle typu
    lineId = vehicle['LineID']
    if lineId <= 12:
        pocty_vozidel_brno['pocet_tramvaji'] += 1
    elif 20 <= lineId <= 39:
        pocty_vozidel_brno['pocet_trolejbusu'] += 1
    elif 40 <= lineId <= 99:
        pocty_vozidel_brno['pocet_autobusu'] += 1

    # Počítání aktivních regionálních vlaků
    if (vehicle['LF']) and ('R' in vehicle['LineName'] or 'S' in vehicle['LineName']):
        pocet_nizkopodlaznich_regionalnich_vlaku += 1

# Výpočty ke zpoždění
procento_zpozdenych_nad5min = pocet_zpozdenych_nad5min / pocet_vozidel * 100
zpozdeni_prumer = zpozdeni_celkem / pocet_vozidel

# Výstupy pro řešení se slovníky
print(f"Celkem vozidel: {pocet_vozidel}")
print(f"Neaktivních vozidel: {pocet_neaktivnich}")
print(f"Vozidel zpožděných více než 5 minut je {pocet_zpozdenych_nad5min} ({procento_zpozdenych_nad5min:.2f} %).")

print(f"Průměrné zpoždění je {zpozdeni_prumer:.2f} minut")
print(f"Počet nízkopodlažních regionálních vlaků je {pocet_nizkopodlaznich_regionalnich_vlaku}")
print(f"Brněnská vozidla: {pocty_vozidel_brno}.")

# Uložení výstupu
with open("pocty_vozidel_slovniky.json", mode="w", encoding="utf-8") as file:
    json.dump(pocty_vozidel_brno, file, ensure_ascii=False, indent=4)

Celkem vozidel: 430
Neaktivních vozidel: 64
Vozidel zpožděných více než 5 minut je 19 (4.42 %).
Průměrné zpoždění je 1.57 minut
Počet nízkopodlažních regionálních vlaků je 19
Brněnská vozidla: {'pocet_tramvaji': 99, 'pocet_trolejbusu': 42, 'pocet_autobusu': 90}.


## 2. Řešení pandas: 

In [9]:
import pandas

vehicles_df = pandas.DataFrame.from_dict(data['Vehicles'])
vehicles_df.head()

,ID,IDB,IDC,VType,LType,Lat,Lng,Bearing,LineID,LineName,RouteID,ServiceId,Course,LF,Delay,LastStopID,FinalStopID,FinalStopName,IsInactive
0,1922,0,0,0,0,49.192734,16.572811,-1,2,2,1505,212,00212,True,0,9410,1343,Lipová,True
1,21571,0,0,4,4,48.853413,17.109982,45,901,901,75,90301,90301,True,0,19019,19008,"Hodonín, nem.láz.",False
2,21136,0,0,4,4,48.755608,16.850769,225,571,571,23,57101,57101,True,0,15745,15801,"Valtice, aut. st.",False
3,1925,0,0,0,0,49.191319,16.612240,45,2,2,1268,206,00206,True,0,1146,1611,Stará osada,False
4,1094,1523,0,0,0,49.231365,16.577042,315,12,12,1306,1221,01221,True,2,1659,1659,Technologický park,False


In [10]:
# Počet všech vozidel
# pocet_vozidel = vehicles_df.shape[0]
pocet_vozidel = len(vehicles_df)

# Počet vozidel, která mají pauzu
neaktivni_vozidla = vehicles_df[vehicles_df["IsInactive"]]
pocet_neaktivnich = neaktivni_vozidla.shape[0]
# pocet_neaktivnich = vehicles_df["IsInactive"].sum()

# Počet vozidel se zpožděním větším než 5 minut
zpozdena_nad5min = vehicles_df[vehicles_df["Delay"] > 5]
pocet_zpozdenych_nad5min = zpozdena_nad5min.shape[0]
procento_zpozdenych_nad5min = pocet_zpozdenych_nad5min / pocet_vozidel * 100

# Průměr zpoždění
zpozdeni_celkem = vehicles_df["Delay"].sum()
zpozdeni_prumer = zpozdeni_celkem / pocet_vozidel

# Počet nízkopodlažních regionálních vlaků
# nizkopodlazni_regionalni_vlaky = vehicles_df[(vehicles_df["LF"]) & (vehicles_df["LineName"].str.contains("R") | vehicles_df["LineName"].str.contains("S"))].shape[0]
nizkopodlazni_regionalni_vlaky = vehicles_df[(vehicles_df["LF"]) & (vehicles_df["LineName"].str.contains("R|S"))]
pocet_nizkopodlaznich_regionalnich_vlaku = nizkopodlazni_regionalni_vlaky.shape[0]


# Výběr vozidel podle typu - složená podmínka
# tramvaje = vehicles_df[(vehicles_df['LineID'] >= 1) & (vehicles_df['LineID'] <= 12)]
# trolejbusy = vehicles_df[(vehicles_df['LineID'] >= 20) & (vehicles_df['LineID'] <= 39)]
# autobusy = vehicles_df[(vehicles_df['LineID'] >= 40) & (vehicles_df['LineID'] <= 99)]

# Výběr vozidel podle typu - metoda between
tramvaje = vehicles_df[vehicles_df['LineID'].between(1,12)]
trolejbusy = vehicles_df[vehicles_df['LineID'].between(20,39)]
autobusy = vehicles_df[vehicles_df['LineID'].between(40,99)]

pocty_vozidel_brno = {
    "pocet_tramvaji": len(tramvaje),
    "pocet_trolejbusu": len(trolejbusy),
    "pocet_autobusu": len(autobusy),
}
# ... případně lze rovnou sčítat řádky splňující podmínku jako např. vehicles_df['LineID'].between(1,12).sum()


# Výstupy pro řešení s pandas
print(f"Celkem vozidel: {pocet_vozidel}")
print(f"Neaktivních vozidel: {pocet_neaktivnich}")
print(f"Vozidel zpožděných více než 5 minut je {pocet_zpozdenych_nad5min} ({procento_zpozdenych_nad5min:.2f} %).")

print(f"Průměrné zpoždění je {zpozdeni_prumer:.2f} minut")
print(f"Počet nízkopodlažních regionálních vlaků je {pocet_nizkopodlaznich_regionalnich_vlaku}")
print(f"Brněnská vozidla: {pocty_vozidel_brno}.")

# Uložení výstupu
with open("pocty_vozidel_pandas.json", mode="w", encoding="utf-8") as file:
    json.dump(pocty_vozidel_brno, file, ensure_ascii=False, indent=4)

Celkem vozidel: 430
Neaktivních vozidel: 64
Vozidel zpožděných více než 5 minut je 19 (4.42 %).
Průměrné zpoždění je 1.57 minut
Počet nízkopodlažních regionálních vlaků je 19
Brněnská vozidla: {'pocet_tramvaji': 99, 'pocet_trolejbusu': 42, 'pocet_autobusu': 90}.
